# Assignment 1: Building a simple image search algorithm
Visual analytics, CDS 2024, Laura Givskov Rahbek 
1. Work with larger datasets of images
2. Extract structured information from image data using ```OpenCV```
3. Quantaitively compare images based on these features, performing *distant viewing*

### Set up and import of packages 

In [1]:
import os 
import sys 

sys.path.append(os.path.join("..", ".."))
import cv2
import numpy as np 
from utils.imutils import jimshow as show 
from utils.imutils import jimshow_channel as show_channel
import matplotlib.pyplot as plt

import glob
import pandas as pd

In [ ]:
# sudo apt-get update
# sudo apt-get install -y python3-opencv
# pip install opencv-python matplotlib
# pip install pandas 

### Define the base-flower (flower 1 or f1)
1. Define filepath 
2. Read in f1 using cv2 and define filename
3. Extract and normalize the colour histogram for the f1

In [6]:
filepath_f1 = os.path.join("..", "..", "..", "..", "cds-vis-data", "flowers", "image_0001.jpg")
image_f1 = cv2.imread(filepath_f1)
filename_f1 = filepath_f1.split("/")[-1]
hist_f1 = cv2.calcHist([image_f1], channels = [0,1,2], mask = None, histSize = [255,255,255], ranges = [0,256, 0,256,0,256])
norm_hist_f11 = cv2.normalize(hist_f1, hist_f1, 0, 1.0, cv2.NORM_MINMAX)

In [7]:
def image_hist(image):
    hist = cv2.calcHist([image], channels = [0,1,2], mask = None, histSize = [255,255,255], ranges = [0,256, 0,256,0,256])
    norm_hist = cv2.normalize(hist, hist, 0, 1.0, cv2.NORM_MINMAX)
    return norm_hist

In [8]:
#function to load in flower to compareand define hist etc 
def pick_flower(image_name):
    filepath_f1 = os.path.join("..", "in", "flowers", image_name)
    image_f1 = cv2.imread(filepath_f1)
    filename_f1 = filepath_f1.split("/")[-1].split(".jpg")[0]
    norm_hist_f1 = image_hist(image_f1)
    outpath_f1 = os.path.join("..", "out", f"{filename_f1}_dist.csv")
    return outpath_f1, norm_hist_f1

In [9]:
outpath_f1, norm_hist_f1 = pick_flower("image_0001.jpg")

In [11]:
def compare_flower(outpath_f1, norm_hist_f1):
    filepath_f = os.path.join("..", "in", "flowers")
    dist_df = pd.DataFrame(columns= ("Filename", "Distance"))

    for file in sorted(os.listdir(filepath_f)):
        filepath_fx = os.path.join(filepath_f, file)
        image_fx = cv2.imread(filepath_fx)
        filename_fx = file.split(".jpg")[0]
        norm_hist_fx = image_hist(image_fx)
        
        dist_fx = round(cv2.compareHist(norm_hist_f1, norm_hist_fx, cv2.HISTCMP_CHISQR),5)
        row_fx = [filename_fx, dist_fx]
        
        if len(dist_df.index)<6:
            dist_df.loc[len(dist_df)] = row_fx
        else:
            max_dist_row = dist_df['Distance'].idxmax()
            max_dist_value = dist_df.loc[max_dist_row, 'Distance']
            if dist_fx<max_dist_value:
                dist_df.loc[max_dist_row] = row_fx

    dist_df.to_csv(outpath_f1) 

In [12]:
compare_flower(outpath_f1, norm_hist_f1)

### Compare f1's histogram to all images' histogram in the data
1. Define filepath to the flowers folder. 
2. Define a dataframe and an outpath where the five most similar filenames and distances are saved.
3. For each image (fx) the following is done:  
    - The image is loaded in and the filename is defined 
    - The histogram is extracted and normalised 
    - The distance between f1 and fx is measured and saved using the chisquared metric
    - If there are less than 6 rows (five most similar to f1 and f1) the filename and distance is appended to the dataframe
    - If there are 6 rows, the maximum distance is compared to the distance of fx, if fx is smaller (closer to f1) it is inserted and the maximum distance row is dropped. 
4. Finally, the data frame with the five most similar images to f1, their filenames and distances, are saved in the out folder


In [3]:
filepath_f = os.path.join("..", "..", "..", "..", "cds-vis-data", "flowers")
distances = pd.DataFrame(columns= ("Filename", "Distance"))
outpath = os.path.join("..", "out", "distances1.csv")

In [4]:
for file in sorted(os.listdir(filepath_f)):
    filepath_fx = os.path.join(filepath_f, file)
    image_fx = cv2.imread(filepath_fx)
    filename_fx = file.split(".jpg")[0]

    hist_fx = cv2.calcHist([image_fx], channels = [0,1,2], mask = None, histSize = [255,255,255], ranges = [0,256, 0,256,0,256])
    norm_hist_fx = cv2.normalize(hist_fx, hist_fx, 0, 1.0, cv2.NORM_MINMAX)

    dist_fx = round(cv2.compareHist(norm_hist_f1, norm_hist_fx, cv2.HISTCMP_CHISQR),5)
    
    row_fx = [filename_fx, dist_fx]
        
    if len(distances.index)<6:
        distances.loc[len(distances)] = row_fx
                       
    else:
        max_dist_row = distances['Distance'].idxmax()
        max_dist_value = distances.loc[max_dist_row, 'Distance']

        if dist_fx<max_dist_value:
            distances.loc[max_dist_row] = row_fx

distances.to_csv(outpath) 

KeyboardInterrupt: 